In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
#langsmith tracking
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ['LANGCHAIN_PROJECT']=os.getenv("LANGCHAIN_PROJECT")

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x106b91a60> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x106b8d1f0> root_client=<openai.OpenAI object at 0x106b916d0> root_async_client=<openai.AsyncOpenAI object at 0x10d07ebb0> openai_api_key=SecretStr('**********') openai_proxy=''


In [ ]:
result = llm.invoke("What is Generative AI?")
print(result)

In [4]:
# Chatprompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a Rizz master. Help me rizz girl online in chat"),
        ("user", "{input}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a Rizz master. Help me rizz girl online in chat')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [ ]:
chain = prompt|llm

response = chain.invoke({"input":"Can you tell me about Langsmith?"})
print(response)

In [ ]:
#string output parser
from langchain_core.output_parsers import StrOutputParser
outparser= StrOutputParser()
chain = prompt|llm|outparser

response = chain.invoke({"input":"Can you tell me about Langsmith?"})
print(response)

## GenAI app

In [6]:
from langchain_community.document_loaders import WebBaseLoader

In [7]:
loader = WebBaseLoader("https://huggingface.co/learn/agents-course/unit2/langgraph/building_blocks")
loader

In [8]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://huggingface.co/learn/agents-course/unit2/langgraph/building_blocks', 'title': 'Building Blocks of LangGraph - Hugging Face Agents Course', 'description': 'We’re on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBuilding Blocks of LangGraph - Hugging Face Agents Course\n\n\n\n\n\n\n\n\nHugging Face\n\n\n\n\n\n\n\n\t\t\t\t\t\tModels\n\n\t\t\t\t\t\tDatasets\n\n\t\t\t\t\t\tSpaces\n\n\n\n\t\t\tCommunity\n\t\t\n\n\n\t\t\t\t\t\tDocs\n\n\t\t\t\t\t\tEnterprise\n\nPricing\n\t\t\t\n\n\n\n\n\n\nLog In\n\t\t\t\t\nSign Up\n\t\t\t\t\t\n\n\n\nAgents Course documentation\n\t\t\t\nBuilding Blocks of LangGraph\n\n\n\nAgents Course\n\n🏡 View all resourcesAgents CourseAudio CourseCommunity Computer Vision CourseDeep RL CourseDiffusion CourseLLM CourseMCP CourseML for 3D CourseML for Games CourseOpen-Source AI 

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)

In [10]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [11]:
from langchain_community.vectorstores import FAISS
vectorDB = FAISS.from_documents(documents, embeddings)
vectorDB

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
querry = "What the major components of LangGraph?"
result = vectorDB.similarity_search(querry)
result[0].page_content

In [ ]:
# Retrieval Chain, Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following questions based only on the provided context:
    <context>
    {context}
    </context>
    """
)
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input": "LangGraph is used for which type of development project?",
    "context":[Document(page_content="LangGraph is used for which type of development project?")]
})

In [ ]:
# input --> Retriever --> vectorDB
vectorDB

NameError: name 'vectorstoreDB' is not defined

In [ ]:
retriever = vectorDB.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [ ]:
response = retrieval_chain.invoke({"input":"Langchain has twoadafae"})
response['answer']

In [ ]:
response['context']